In [137]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
from torch.autograd import Variable
import matplotlib.pyplot as plt
import seaborn
seaborn.set_context(context="talk")
%matplotlib inline

In [144]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [143]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

In [32]:
class EncoderDecoder(nn.Module):
    """
    A standard Encoder-Decoder architecture. Base for this and many
    other models.
    """
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def forward(self, src, tgt, src_mask, tgt_mask):
        "Take in and process masked src and target sequences."
        return self.decode(self.encode(src, src_mask), src_mask,
                            tgt, tgt_mask)

    def encode(self, src, src_mask):
    
        return self.encoder(self.src_embed(src).to(device), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt).to(device), memory, src_mask, tgt_mask)

In [33]:
class Generator(nn.Module):
    "Define standard linear + softmax generation step."
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return self.proj(x) 

In [34]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [35]:
class Encoder(nn.Module):
    "Core encoder is a stack of N layers"
    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        "Pass the input (and mask) through each layer in turn."
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [36]:
class LayerNorm(nn.Module):
    "Construct a layernorm module (See citation for details)."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        """ Return a2 * x_normalized + b2,
        where x_normalized is calculated by subtracting row-wise means from x and dividing the result by row-wise standard deviation + eps.
        standard deviation is calculated with Bessel's correction (the default in Pytorch)
        """

        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [37]:
class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))

In [38]:
class EncoderLayer(nn.Module):
    "Encoder is made up of self-attn and feed forward (defined below)"
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        "Follow Figure 1 (left) for connections."
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

In [39]:
class Decoder(nn.Module):
    "Generic N layer decoder with masking."
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)

In [40]:
class DecoderLayer(nn.Module):
    "Decoder is made of self-attn, src-attn, and feed forward (defined below)"
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Follow Figure 1 (right) for connections."
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)

In [41]:
def subsequent_mask(size):
    "Mask out subsequent positions."
    attn_shape = (1, size, size)
    subsequent_mask = np.triu(np.ones(attn_shape), k=1).astype('uint8')
    return torch.from_numpy(subsequent_mask) == 0

In [42]:
def attention(query, key, value, mask=None, dropout=None):
    """Compute 'Scaled Dot Product Attention'
    """
    d_k = query.size(-1)
    # todo: compute the attention scores by using torch.matmul
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:

        scores = scores.masked_fill(mask == 0, -1e9)
    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    # todo: compute the result as the values weighted by attention probabilities (again, using torch.matmul)
    result = torch.matmul(p_attn, value)
    return result, p_attn

In [43]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        "Implements Figure 2"
        if mask is not None:
            # Same mask applied to all h heads.
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Do all the linear projections in batch from d_model => h x d_k
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (query, key, value))]

        # 2) Apply attention on all the projected vectors in batch.
        x, self.attn = attention(query, key, value, mask=mask,
                                 dropout=self.dropout)

        # 3) "Concat" using a view and apply a final linear.
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

In [44]:
class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))

In [118]:
class Embeddings(nn.Module):
    def __init__(self, d_model, d_vocab):
        super().__init__()
        
        self.transform = nn.Linear(d_vocab, d_model, dtype=float).to(device)
        self.d_vocab = d_vocab
        self.d_model = d_model

    def forward(self, x):
        len_ = int(len(x[0]) / self.d_vocab )
        trajs_emb = torch.zeros(len(x), len_ , self.d_model, requires_grad=True).to(device)
        
        for j, x_tmp in enumerate(x):
            traj_emb = torch.zeros(len_ , self.d_model, requires_grad=False).to(device)
            
            for i in range(self.d_vocab, len_ + 1, self.d_vocab):
                point = x_tmp[i - self.d_vocab:i]          
                traj_emb[i - self.d_vocab, :] =  self.transform(point)
            trajs_emb[j,:,:] =  traj_emb

        return trajs_emb * math.sqrt(self.d_model)

In [51]:
class PositionalEncoding(nn.Module):
    "Implement the PE function."
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + Variable(self.pe[:, :x.size(1)], requires_grad=False)
        return self.dropout(x)

In [52]:
def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    "Helper: Construct a model from hyperparameters."
    c = copy.deepcopy  # use it for attn, ffn, and position in the model layers
    attn = MultiHeadedAttention(h, d_model)
    ff = PositionwiseFeedForward(d_model, d_ff, dropout)
    position = PositionalEncoding(d_model, dropout)
    # todo: insert correct arguments into the EncoderDecoder constructor.
    model = EncoderDecoder(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab)
    )

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

In [53]:

class NoamOpt:
    "Optim wrapper that implements rate."
    def __init__(self, model_size, factor, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.factor = factor
        self.model_size = model_size
        self._rate = 0

    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()

    def rate(self, step = None):
        "Implement `lrate` above"
        if step is None:
            step = self._step
        return self.factor * \
            (self.model_size ** (-0.5) *
            min(step ** (-0.5), step * self.warmup ** (-1.5)))

def get_std_opt(model):
    return NoamOpt(model.src_embed[0].d_model, 2, 4000,
            torch.optim.Adam(model.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9))

In [54]:
def loss_backprop(generator, criterion, out, targets):
    """
    Memory optmization. Compute each timestep separately and sum grads.
    """
    assert out.size(1) == targets.size(1)
    targets = targets.to(torch.float32)
    
    total = 0.0
    out_grad = []
    for i in range(out.size(1)):
        out_column = Variable(out[:, i].data, requires_grad=True)
        gen = generator(out_column)
        loss = criterion(gen, targets[:, i]) 
        total += loss.item() #.data[0]
        loss.backward()
        out_grad.append(out_column.grad.data.clone())
    out_grad = torch.stack(out_grad, dim=1)
    out.backward(gradient=out_grad)
    return total


In [55]:
def loss_(generator, criterion, out, targets):
    """
    Memory optmization. Compute each timestep separately and sum grads.
    """
    assert out.size(1) == targets.size(1)
    targets = targets.to(torch.float32)

          
    total = 0.0
    out_grad = []
    for i in range(out.size(1)):
        out_column = Variable(out[:, i].data, requires_grad=True)
        gen = generator(out_column)
        loss = criterion(gen, targets[:, i]) 
        total += loss.item() #.data[0]
        
    return total


In [56]:
def make_std_mask(src, tgt, pad=-1):
    
    src_mask = (src[-1,:int(src.shape[1]/2)] != pad).unsqueeze(-2)
    tgt_mask = (tgt != pad).unsqueeze(-2)
    tgt_mask = tgt_mask & Variable(subsequent_mask(tgt.size(-1)).type_as(tgt_mask.data))
    return src_mask, tgt_mask

In [57]:
def train_epoch(train_iter, model, criterion, opt, transpose=False):
    model.train()
    for i, batch in enumerate(train_iter):
        src, trg, src_mask, trg_mask = \
                batch.src.to(device), batch.trg.to(device), batch.src_mask.to(device), batch.trg_mask.to(device)
        
        out = model.forward(src, trg, src_mask, trg_mask)
        loss = loss_backprop(model.generator, criterion, out, trg)

        model_opt.step()
        model_opt.optimizer.zero_grad()

    print('test loss: ', loss)

In [58]:
def valid_epoch(valid_iter, model, criterion, transpose=False):
    model.eval()
    total = 0
    for batch in valid_iter:
        src, trg, src_mask, trg_mask = \
            batch.src.to(device), batch.trg.to(device), batch.src_mask.to(device), batch.trg_mask.to(device)
        out = model.forward(src, trg, src_mask, trg_mask)
        loss = loss_(model.generator, criterion, out, trg)
    print('validation loss: ', loss)

# ____

In [59]:
class Batch:
    def __init__(self, src, trg, src_mask, trg_mask):
        self.src = src
        self.trg = trg
        self.src_mask = src_mask
        self.trg_mask = trg_mask


In [60]:
import random

def data_iterator(srcs, tgts, batch_size=128, shuffle=True):
    if shuffle:
        pairs = list(zip(srcs, tgts))
        random.shuffle(pairs)
        srcs, tgts = [list(t) for t in zip(*pairs)]

    for i in range(0, len(srcs), batch_size):
        x = torch.tensor(srcs[i: i + batch_size])
        y = torch.tensor(tgts[i: i + batch_size])
        src = Variable(x, requires_grad=False)
        tgt = Variable(y, requires_grad=False)
        src_mask, tgt_mask = make_std_mask(src, tgt, -1)
        yield Batch(src, tgt, src_mask, tgt_mask)

In [61]:
with open('trajs.csv', 'rb') as f:
    trajs_train = np.load(f)
    
with open('labels.csv', 'rb') as f:
    labels_train = np.load(f)
    
with open('trajs_test.csv', 'rb') as f:
    trajs_test = np.load(f)
    
with open('labels_test.csv', 'rb') as f:
    labels_test = np.load(f)


In [62]:

N, d, T = trajs_train.shape 

trajs_ = np.zeros((N, d * T + 2))

for i in range(N):
    for j in range(T):
        trajs_[i , d * j + 2] = trajs_train[i, 0, j]
        trajs_[i , d * j + 1 + 2] = trajs_train[i, 1, j]
        
trajs_train = trajs_
 

In [63]:

N, d, T = trajs_test.shape 

trajs_ = np.zeros((N, d * T + 2))

for i in range(N):
    for j in range(T):
        trajs_[i , d * j + 2] = trajs_test[i, 0, j]
        trajs_[i , d * j + 1 + 2] = trajs_test[i, 1, j]

trajs_test = trajs_

In [64]:

N, _, T = labels_train.shape

labels_ = np.zeros((N, T + 1))

labels_[:, 1:] =  labels_train[:,0, :]

labels_train = labels_


In [65]:

N, _, T = labels_test.shape

labels_ = np.zeros((N, T + 1))

labels_[:, 1:] =  labels_test[:,0, :]

labels_test = labels_


In [119]:

model = make_model(2, 1, N=6, d_model=256, d_ff=1023, h=4, dropout=0.1)
model_opt = get_std_opt(model)

criterion = nn.MSELoss()
criterion.to(device)

model.to(device)

<ipython-input-52-ef8cf4ad348d>:20: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


EncoderDecoder(
  (encoder): Encoder(
    (layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0-3): 4 x Linear(in_features=256, out_features=256, bias=True)
          )
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=256, out_features=1023, bias=True)
          (w_2): Linear(in_features=1023, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (sublayer): ModuleList(
          (0-1): 2 x SublayerConnection(
            (norm): LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
      )
    )
    (norm): LayerNorm()
  )
  (decoder): Decoder(
    (layers): ModuleList(
      (0-5): 6 x DecoderLayer(
        (self_attn): MultiHeadedAttention(
          (linears): ModuleList(
            (0-3): 4 x Linear(in_features=256, out_

In [120]:
def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 11,055,605 trainable parameters


In [121]:
BATCH_SIZE = 2

In [122]:
from tqdm import tqdm

In [129]:
epoches = 100

model.to(device)
for epoch in tqdm(range(epoches)):
    print(epoch,' epoch')
    train_epoch(data_iterator(trajs_train, labels_train, batch_size=BATCH_SIZE), model, criterion, model_opt)
    valid_epoch(data_iterator(trajs_test, labels_test, batch_size=BATCH_SIZE), model, criterion)

  0%|          | 0/100 [00:00<?, ?it/s]

0  epoch
test loss:  12.353393045719713


  1%|          | 1/100 [02:06<3:28:56, 126.63s/it]

validation loss:  8.596023071557283
1  epoch
test loss:  11.056173251302198


  2%|▏         | 2/100 [04:14<3:27:38, 127.13s/it]

...

validation loss:  10.066652244742727
98  epoch
test loss:  7.347303916298188


 99%|█████████▉| 99/100 [3:33:34<02:09, 129.48s/it]

validation loss:  5.1578006982226725
99  epoch
test loss:  11.085944295704394


100%|██████████| 100/100 [3:35:44<00:00, 129.45s/it]

validation loss:  11.106605242477599


# _______

In [130]:
def prediction(model, src, src_mask, max_len, start_symbol):
    
    memory = model.encode(src.to(device), src_mask.to(device))
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    
    pred = torch.zeros(src.shape)

    for i in range(max_len-1):
        out = model.decode(memory, src_mask.to(device),
                           Variable(ys).to(device),
                           Variable(subsequent_mask(ys.size(1))
                                    .type_as(src.data).to(device)))
        gen = model.generator(out[:, -1])
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(*gen[0])], dim=1)
    return ys


In [135]:
n = 21

for batch in data_iterator(trajs_test[n:n+1], labels_test[n:n+1], batch_size=1):
    
    src, trg, src_mask, trg_mask = \
            batch.src.to(device), batch.trg.to(device), batch.src_mask.to(device), batch.trg_mask.to(device)


In [140]:
model.eval()
model.to(device)
max_len = 201
start_symbol = 0

pred = prediction(model, src, src_mask, max_len, start_symbol)

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 MiB. GPU 0 has a total capacity of 10.90 GiB of which 1.31 MiB is free. Including non-PyTorch memory, this process has 10.90 GiB memory in use. Of the allocated memory 10.64 GiB is allocated by PyTorch, and 85.98 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

100 эпох

In [133]:
pred

tensor([[0.0000, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0041, 0.0041,
         0.0041, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0041, 0.0041, 0.0041,
         0.0041, 0.0041, 0.0041, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040, 0.0040,
         0.0040, 0.0040, 0.0

In [134]:
trg

tensor([[0.0000, 1.2000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000,
         0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000,
         0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000,
         0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 0.7000,
         0.7000, 0.7000, 0.7000, 0.7000, 0.7000, 1.2000, 1.2000, 0.7000, 0.7000,
         0.7000, 0.7000, 0.7000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000, 1.2000,
         1.2000, 1.2000, 1.2

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

pred = pred.detach()

f, axes = plt.subplots(1, 1, sharex=True, figsize=(14, 5))



axes.plot(np.linspace(0, 200, 200), pred, label='prediction')
axes.plot(x_np, x.grad, label=function_name)

x.grad.zero_()

### END Solution (do not delete this comment)

axes[i, 0].legend()
axes[i, 1].legend()

plt.tight_layout()
plt.show()